<a href="https://colab.research.google.com/github/abdullahmujahidali/Vet-LangGraph/blob/main/VetAI_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q langchain langgraph langchain_openai openai python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.0/150.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.0/55.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.0/413.0 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.4 MB/s eta 0:00:00


In [2]:
import os
from typing import List, Dict, Tuple, Annotated, TypedDict, Union, Any, Optional
from google.colab import userdata
from datetime import datetime
from enum import Enum
from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, AIMessage
import json


class DiagnosisType(str, Enum):
    DEFINITIVE = "definitive"
    DIFFERENTIAL = "differential"
    SYMPTOM_BASED = "symptom_based"
    LAB_ABNORMALITY = "lab_abnormality"

class ConfidenceLevel(str, Enum):
    CONFIRMED = "confirmed"
    PRESUMPTIVE = "presumptive"
    OWNER_REPORTED = "owner_reported"

class ProblemStatus(str, Enum):
    ACTIVE = "active"
    HISTORICAL = "historical"
    RESOLVED = "resolved"
    RELAPSED = "relapsed"

class TreatmentResponse(str, Enum):
    IMPROVED = "improved"
    WORSENED = "worsened"
    NO_CHANGE = "no_change"
    SIDE_EFFECTS = "side_effects"

class Medication(TypedDict):
    drug_name: str
    formulation: str
    concentration: str
    dose_given: str
    frequency: str
    duration: str
    calculated_dose_mg_per_kg: Optional[float]
    response: TreatmentResponse
    owner_perception: str
    side_effects: str

class Problem(TypedDict):
    description: str
    type: DiagnosisType
    confidence: ConfidenceLevel
    status: ProblemStatus
    onset_date: str
    progression: str
    related_symptoms: List[str]
    treatments: List[Medication]

class LabResult(TypedDict):
    test_name: str
    value: str
    reference_range: str
    unit: str
    flag: str
    date: str

class VetState(TypedDict):
    messages: List[Union[HumanMessage, AIMessage]]
    current_input: str

    medical_records: Dict[str, Any]
    owner_history: Dict[str, Any]
    diagnoses: Dict[str, Problem]
    lab_results: Dict[str, LabResult]

    medications: Dict[str, Medication]
    treatment_history: Dict[str, Any]

    results: Dict[str, Any]

    metadata: Dict[str, Any]
    security_flags: Dict[str, Any]

    calculations: Dict[str, Any]
    trends: Dict[str, Any]

    owner_constraints: Dict[str, Any]
    owner_preferences: Dict[str, Any]



OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0
)

In [3]:
def process_data(state: VetState) -> VetState:
    prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a veterinary data processing agent responsible for initial data extraction and organization.

KEY RESPONSIBILITIES:
1. File Processing:
   - Process veterinary records, lab reports, and supporting documents
   - Extract and structure content based on document type
   - Apply standardized formatting
   - Generate metadata tags for indexing

2. Content Organization:
   - Patient Information (Name, Species, Breed, Age, ID)
   - Clinical History & Physical Exam findings
   - SOAP Notes and Progress Reports
   - Medications & Treatments (with precise dosing)
   - Laboratory Results (with reference ranges)
   - Diagnostic Imaging Reports
   - Doctor's Recommendations

3. Data Security:
   - Identify and flag sensitive information
   - Apply appropriate redaction where needed
   - Note areas requiring manual review

OUTPUT FORMAT:
Generate a structured output containing:
1. Parsed medical data in standardized format
2. Metadata tags for indexing
3. Security/privacy flags
4. Data quality indicators"""),
        ("human", "Process and structure the following veterinary input: {input}")
    ])

    try:
        messages = prompt.format_messages(input=state["current_input"])
        response = llm.invoke(messages)

        new_state = state.copy()
        new_state["messages"].append(response)
        new_state["results"]["DataProcessor"] = response.content
        new_state["metadata"]["processing_timestamp"] = datetime.now().isoformat()

        return new_state

    except Exception as e:
        return handle_error(state, f"Data Processing Error: {str(e)}")

def analyze_history(state: VetState) -> VetState:
    prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a veterinary history analysis agent specializing in comprehensive medical history evaluation.

KEY RESPONSIBILITIES:
1. Medical History Analysis:
   - Extract all past and current medical conditions
   - Categorize by body system
   - Document onset dates, severity, progression
   - Calculate precise medication dosing

2. Problem Categorization:
   - Active vs Historical Problems
   - Related vs Separate Issues
   - Primary vs Secondary Conditions
   - Track progression status

3. Treatment History:
   - Document all medications with exact details
   - Calculate mg/kg dosing where applicable
   - Track treatment responses
   - Note adverse reactions

4. Owner Information:
   - Document reported symptoms
   - Note owner concerns
   - Track compliance history
   - Record practical limitations"""),
        ("human", "Based on this processed data, provide a comprehensive medical history analysis: {processed_data}")
    ])

    try:
        processed_data = state["results"]["DataProcessor"]
        messages = prompt.format_messages(processed_data=processed_data)
        response = llm.invoke(messages)

        new_state = state.copy()
        new_state["messages"].append(response)
        new_state["results"]["HistoryAnalyzer"] = response.content

        return new_state

    except Exception as e:
        return handle_error(state, f"History Analysis Error: {str(e)}")

def analyze_clinical(state: VetState) -> VetState:
    prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a veterinary clinical record analyst responsible for comprehensive medical record review and synthesis.

KEY RESPONSIBILITIES:
1. Record Analysis:
   - Review all medical documentation
   - Track patient details and trends
   - Analyze laboratory results
   - Document owner compliance

2. Data Trending:
   - Track weight and BCS changes
   - Monitor lab value trends
   - Document treatment responses
   - Note disease progression

3. Pattern Recognition:
   - Identify related symptoms
   - Track disease progression
   - Note treatment responses
   - Flag unusual presentations"""),
        ("human", "Based on this history analysis, provide a detailed clinical analysis: {history_analysis}")
    ])

    try:
        history_analysis = state["results"]["HistoryAnalyzer"]
        messages = prompt.format_messages(history_analysis=history_analysis)
        response = llm.invoke(messages)

        new_state = state.copy()
        new_state["messages"].append(response)
        new_state["results"]["ClinicalAnalyzer"] = response.content

        return new_state

    except Exception as e:
        return handle_error(state, f"Clinical Analysis Error: {str(e)}")

def diagnose(state: VetState) -> VetState:
    prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a veterinary diagnostic intelligence agent responsible for analyzing and categorizing medical problems with confidence levels.

KEY RESPONSIBILITIES:
1. Problem Classification:
   - Categorize diagnoses by type
   - Assign confidence levels
   - Document progression patterns
   - Identify related issues

2. Diagnostic Analysis:
   - Cross-reference symptoms
   - Evaluate test results
   - Consider breed-specific issues
   - Factor in age-related concerns"""),
        ("human", "Based on this clinical analysis, provide diagnostic insights and recommendations: {clinical_analysis}")
    ])

    try:
        clinical_analysis = state["results"]["ClinicalAnalyzer"]
        messages = prompt.format_messages(clinical_analysis=clinical_analysis)
        response = llm.invoke(messages)

        new_state = state.copy()
        new_state["messages"].append(response)
        new_state["results"]["Diagnostics"] = response.content

        return new_state

    except Exception as e:
        return handle_error(state, f"Diagnostic Error: {str(e)}")


def handle_error(state: VetState, error_message: str) -> VetState:
    new_state = state.copy()
    if "errors" not in new_state["results"]:
        new_state["results"]["errors"] = []

    new_state["results"]["errors"].append({
        "timestamp": datetime.now().isoformat(),
        "message": error_message
    })

    return new_state

def calculate_medication_dosing(medications: List[Dict], weight: Optional[float]) -> Dict:
    if not weight:
        return {"error": "No weight available for dosing calculations"}

    results = {}
    for med in medications:
        try:
            dose_str = med.get("dose_given", "")
            dose_num = float(''.join(filter(str.isdigit, dose_str)))

            mg_per_kg = dose_num / weight

            results[med["drug_name"]] = {
                "mg_per_kg": round(mg_per_kg, 2),
                "calculated_from": {
                    "dose": dose_num,
                    "weight": weight
                }
            }
        except Exception as e:
            results[med["drug_name"]] = {
                "error": f"Calculation failed: {str(e)}"
            }

    return results
def create_data_processing_agent():
    prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a veterinary data processing agent responsible for initial data extraction and organization.

KEY RESPONSIBILITIES:
1. File Processing:
   - Process veterinary records, lab reports, and supporting documents
   - Extract and structure content based on document type
   - Apply standardized formatting
   - Generate metadata tags for indexing

2. Content Organization:
   - Patient Information (Name, Species, Breed, Age, ID)
   - Clinical History & Physical Exam findings
   - SOAP Notes and Progress Reports
   - Medications & Treatments (with precise dosing)
   - Laboratory Results (with reference ranges)
   - Diagnostic Imaging Reports
   - Doctor's Recommendations

3. Data Security:
   - Identify and flag sensitive information
   - Apply appropriate redaction where needed
   - Note areas requiring manual review"""),
        ("human", "Process and structure the following veterinary input: {input}")
    ])

    def process_data(state: VetState):
        try:
            messages = prompt.format_messages(input=state["current_input"])
            response = llm.invoke(messages)

            new_state = state.copy()
            new_state["messages"].append(response)
            new_state["results"]["DataProcessor"] = response.content
            new_state["metadata"]["processing_timestamp"] = datetime.now().isoformat()

            return new_state

        except Exception as e:
            return handle_error(state, f"Data Processing Error: {str(e)}")

    return process_data

def create_history_analysis_agent():
    prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a veterinary history analysis agent specializing in comprehensive medical history evaluation.

KEY RESPONSIBILITIES:
1. Medical History Analysis:
   - Extract all past and current medical conditions
   - Categorize by body system
   - Document onset dates, severity, progression
   - Calculate precise medication dosing

2. Problem Categorization:
   - Active vs Historical Problems
   - Related vs Separate Issues
   - Primary vs Secondary Conditions
   - Track progression status"""),
        ("human", "Based on this processed data, provide a comprehensive medical history analysis: {processed_data}")
    ])

    def analyze_history(state: VetState):
        try:
            processed_data = state["results"]["DataProcessor"]
            messages = prompt.format_messages(processed_data=processed_data)
            response = llm.invoke(messages)

            new_state = state.copy()
            new_state["messages"].append(response)
            new_state["results"]["HistoryAnalyzer"] = response.content

            return new_state

        except Exception as e:
            return handle_error(state, f"History Analysis Error: {str(e)}")

    return analyze_history

def create_clinical_analysis_agent():
    prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a veterinary clinical record analyst responsible for comprehensive medical record review and synthesis.

KEY RESPONSIBILITIES:
1. Record Analysis:
   - Review all medical documentation
   - Track patient details and trends
   - Analyze laboratory results
   - Document owner compliance

2. Data Trending:
   - Track weight and BCS changes
   - Monitor lab value trends
   - Document treatment responses
   - Note disease progression"""),
        ("human", "Based on this history analysis, provide a detailed clinical analysis: {history_analysis}")
    ])

    def analyze_clinical(state: VetState):
        try:
            history_analysis = state["results"]["HistoryAnalyzer"]
            messages = prompt.format_messages(history_analysis=history_analysis)
            response = llm.invoke(messages)

            new_state = state.copy()
            new_state["messages"].append(response)
            new_state["results"]["ClinicalAnalyzer"] = response.content

            return new_state

        except Exception as e:
            return handle_error(state, f"Clinical Analysis Error: {str(e)}")

    return analyze_clinical

def create_diagnostic_agent():
    prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a veterinary diagnostic intelligence agent responsible for analyzing and categorizing medical problems with confidence levels.

KEY RESPONSIBILITIES:
1. Problem Classification:
   - Categorize by type (Definitive, Differential, Symptom-Based)
   - Assign confidence levels
   - Document progression patterns
   - Identify related issues

2. Diagnostic Analysis:
   - Cross-reference symptoms
   - Evaluate test results
   - Consider breed-specific issues
   - Factor in age-related concerns

3. Treatment Impact Assessment:
   - Evaluate therapeutic responses
   - Document treatment failures
   - Track adverse reactions
   - Monitor progression"""),
        ("human", "Based on this clinical analysis, provide diagnostic insights and recommendations: {clinical_analysis}")
    ])

    def diagnose(state: VetState):
        clinical_analysis = state["results"]["ClinicalAnalyzer"]
        messages = prompt.format_messages(clinical_analysis=clinical_analysis)
        response = llm.invoke(messages)

        new_state = state.copy()
        new_state["messages"].append(response)
        new_state["results"]["Diagnostics"] = response.content
        return new_state

    return diagnose

def create_specialist_summary_agent():
    prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a veterinary specialist assessment and owner communication agent.

KEY RESPONSIBILITIES:
1. Specialist Assessment:
   - Provide detailed medical analysis of each problem
   - Define pathophysiology and likely causes
   - Assess severity and progression
   - Evaluate current treatment efficacy
   - Outline potential complications

2. Owner Communication:
   - Create clear, layman-friendly explanations
   - Explain what's wrong in simple terms
   - Outline treatment options and expectations
   - Discuss risks of non-treatment
   - Provide clear next steps

3. Treatment Planning:
   - Recommend first-line therapies
   - Suggest adjunctive treatments
   - Consider owner constraints
   - Provide monitoring criteria

OUTPUT FORMAT:
For each problem provide:
1. Specialist-Level Assessment
2. Owner-Friendly Summary
3. Treatment Plan
4. Monitoring Strategy"""),
        ("human", "Based on this diagnostic assessment, provide specialist interpretation and owner-friendly summary: {diagnostic_assessment}")
    ])

    def generate_summaries(state: VetState):
        diagnostic_assessment = state["results"]["Diagnostics"]
        messages = prompt.format_messages(diagnostic_assessment=diagnostic_assessment)
        response = llm.invoke(messages)

        new_state = state.copy()
        new_state["messages"].append(response)
        new_state["results"]["SpecialistSummary"] = response.content
        return new_state

    return generate_summaries

def create_connections_agent():
    prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a veterinary connections and trends analysis agent.

KEY RESPONSIBILITIES:
1. Problem Connection Analysis:
   - Identify primary vs secondary problems
   - Group related symptoms under confirmed diagnoses
   - Flag unexplained symptoms
   - Detect pattern progression

2. Case Theory Development:
   - Construct unified case theories
   - Identify underlying causes
   - Note gaps requiring investigation
   - Suggest focused workups

3. Treatment Integration:
   - Evaluate overall treatment strategy
   - Identify potential interactions
   - Suggest treatment prioritization
   - Note treatment conflicts

OUTPUT FORMAT:
1. Primary/Secondary Problem Classification
2. Unexplained Symptoms List
3. Unified Case Theory
4. Recommended Next Steps"""),
        ("human", "Based on all previous analyses, identify connections and develop a case theory: {previous_analyses}")
    ])

    def analyze_connections(state: VetState):
        previous_analyses = {
            "data_processing": state["results"]["DataProcessor"],
            "history": state["results"]["HistoryAnalyzer"],
            "clinical": state["results"]["ClinicalAnalyzer"],
            "diagnostics": state["results"]["Diagnostics"],
            "specialist": state["results"]["SpecialistSummary"]
        }

        messages = prompt.format_messages(previous_analyses=json.dumps(previous_analyses))
        response = llm.invoke(messages)

        new_state = state.copy()
        new_state["messages"].append(response)
        new_state["results"]["Connections"] = response.content
        return new_state

    return analyze_connections



In [4]:
data_processor = create_data_processing_agent()
history_analyzer = create_history_analysis_agent()
clinical_analyzer = create_clinical_analysis_agent()
diagnostic_agent = create_diagnostic_agent()
specialist_agent = create_specialist_summary_agent()
connections_agent = create_connections_agent()

workflow = StateGraph(VetState)

workflow.add_node("process_data", data_processor)
workflow.add_node("analyze_history", history_analyzer)
workflow.add_node("analyze_clinical", clinical_analyzer)
workflow.add_node("diagnose", diagnostic_agent)
workflow.add_node("specialist_summary", specialist_agent)
workflow.add_node("analyze_connections", connections_agent)

workflow.set_entry_point("process_data")

workflow.add_edge("process_data", "analyze_history")
workflow.add_edge("analyze_history", "analyze_clinical")
workflow.add_edge("analyze_clinical", "diagnose")
workflow.add_edge("diagnose", "specialist_summary")
workflow.add_edge("specialist_summary", "analyze_connections")
workflow.add_edge("analyze_connections", END)

app = workflow.compile()

test_case = {
    "messages": [],
    "current_input": """
    Patient Information:
    Name: Luna
    Species: Feline
    Breed: Domestic Shorthair
    Age: 13 years
    Weight: 4.2 kg (Previous: 4.8 kg 3 months ago)
    Sex: Female Spayed

    History:
    Presenting Complaints:
    - Weight loss despite good appetite
    - Increased thirst and urination
    - Occasional vomiting (2-3 times per week)
    - Vocal at night

    Previous Medical History:
    - Dental cleaning and extraction (2 teeth) in 2023
    - Historical UTI treated in 2022

    Recent Diagnostics:
    - CBC/Chemistry (1 week ago):
      * T4: 62 nmol/L (15-50)
      * ALT: 98 U/L (10-90)
      * BUN: 35 mg/dL (16-36)
      * Creatinine: 1.8 mg/dL (0.8-2.4)
      * Phosphorus: 4.5 mg/dL (3.1-7.5)

    Current Medications:
    - Methimazole 2.5mg BID (started 3 days ago)

    Owner Notes:
    - Lives indoor only
    - Eating well but losing weight
    - More active at night
    - Financial constraints noted
    - Prefers pill form medications (easy to administer)
    """,
    "medical_records": {},
    "owner_history": {},
    "diagnoses": {},
    "results": {},
    "metadata": {},
    "security_flags": {},
    "calculations": {},
    "trends": {},
    "owner_constraints": {},
    "owner_preferences": {}
}

result = app.invoke(test_case)

print("\nData Processing Results:")
print(result["results"]["DataProcessor"])
print("\nHistory Analysis:")
print(result["results"]["HistoryAnalyzer"])
print("\nClinical Analysis:")
print(result["results"]["ClinicalAnalyzer"])
print("\nDiagnostic Assessment:")
print(result["results"]["Diagnostics"])
print("\nSpecialist Summary:")
print(result["results"]["SpecialistSummary"])
print("\nConnections Analysis:")
print(result["results"]["Connections"])


Data Processing Results:
**Patient Information:**
- Name: Luna
- Species: Feline
- Breed: Domestic Shorthair
- Age: 13 years
- Weight: 4.2 kg (Previous: 4.8 kg 3 months ago)
- Sex: Female Spayed

**History:**
**Presenting Complaints:**
- Weight loss despite good appetite
- Increased thirst and urination
- Occasional vomiting (2-3 times per week)
- Vocal at night

**Previous Medical History:**
- Dental cleaning and extraction (2 teeth) in 2023
- Historical UTI treated in 2022

**Recent Diagnostics:**
- CBC/Chemistry (1 week ago):
  - T4: 62 nmol/L (15-50)
  - ALT: 98 U/L (10-90)
  - BUN: 35 mg/dL (16-36)
  - Creatinine: 1.8 mg/dL (0.8-2.4)
  - Phosphorus: 4.5 mg/dL (3.1-7.5)

**Current Medications:**
- Methimazole 2.5mg BID (started 3 days ago)

**Owner Notes:**
- Lives indoor only
- Eating well but losing weight
- More active at night
- Financial constraints noted
- Prefers pill form medications (easy to administer)

History Analysis:
**Comprehensive Medical History Analysis for Luna:

In [5]:
test_case_1 = {
    "messages": [],
    "current_input": """
    Patient Information:
    Name: Whiskers
    Species: Feline
    Breed: Maine Coon
    Age: 14 years
    Weight: 5.2 kg (Previous: 6.1 kg 2 months ago)
    Sex: Male Neutered

    History:
    Presenting Complaints:
    - Progressive weight loss over 2 months
    - Increased water consumption
    - Vomiting 2-3 times per week
    - Decreased grooming
    - Lethargy

    Previous Medical History:
    - Dental cleaning and extractions (3 teeth) in 2023
    - Chronic kidney disease diagnosed 2022
    - UTI treated in 2021

    Recent Diagnostics:
    - CBC/Chemistry (3 days ago):
      * BUN: 45 mg/dL (18-35)
      * Creatinine: 2.8 mg/dL (0.8-2.4)
      * Phosphorus: 6.2 mg/dL (3.4-8.5)
      * T4: 4.8 µg/dL (1.5-4.0)
      * ALT: 85 U/L (10-100)
      * Glucose: 95 mg/dL (71-159)
    - Urinalysis:
      * USG: 1.035
      * pH: 6.5
      * Protein: 2+
    - Blood Pressure: 165/95 mmHg

    Current Medications:
    - Semintra 4mg/ml (0.25ml PO SID)
    - Methimazole 5mg BID
    - Renal diet (Hill's k/d)

    Owner Notes:
    - Lives indoors with one other cat
    - Still eating but less enthusiastic
    - Using litter box more frequently
    - Financial constraints noted
    - Prefers pill form medications
    - Can monitor closely (works from home)
    """,
    "medical_records": {},
    "owner_history": {},
    "diagnoses": {},
    "lab_results": {},
    "medications": {},
    "treatment_history": {},
    "results": {},
    "metadata": {},
    "security_flags": {},
    "calculations": {},
    "trends": {},
    "owner_constraints": {},
    "owner_preferences": {}
}

test_case_2 = {
    "messages": [],
    "current_input": """
    Patient Information:
    Name: Max
    Species: Canine
    Breed: Golden Retriever
    Age: 8 months
    Weight: 25.4 kg
    Sex: Male Intact

    History:
    Presenting Complaints:
    - Acute onset vomiting (started 6 hours ago)
    - Severe lethargy
    - Abdominal pain
    - Last seen chewing corn cob 12 hours ago

    Previous Medical History:
    - Vaccinations up to date
    - Dewormed monthly
    - No previous surgeries

    Physical Examination:
    - Temperature: 103.2°F
    - Heart Rate: 130 bpm
    - Respiratory Rate: 32 breaths/min
    - Painful on abdominal palpation
    - Moderate dehydration (8%)

    Recent Diagnostics:
    - Abdominal radiographs:
      * Foreign body visible in small intestine
      * No free air visible
    - CBC:
      * WBC: 22,000/µL (6,000-17,000)
      * HCT: 58% (37-55)
    - Chemistry:
      * ALT: 45 U/L (10-100)
      * BUN: 42 mg/dL (7-27)

    Current Treatment:
    - IV Catheter placed
    - LRS fluids at 180ml/hr
    - Cerenia 1mg/kg SC
    - Methadone 0.3mg/kg IM

    Owner Notes:
    - Pet insurance active
    - Authorized for emergency surgery
    - Can provide 24/7 post-op care
    - No financial constraints noted
    """,
    "medical_records": {},
    "owner_history": {},
    "diagnoses": {},
    "lab_results": {},
    "medications": {},
    "treatment_history": {},
    "results": {},
    "metadata": {},
    "security_flags": {},
    "calculations": {},
    "trends": {},
    "owner_constraints": {},
    "owner_preferences": {}
}


test_case_3 = {
    "messages": [],
    "current_input": """
    Patient Information:
    Name: Milo
    Species: Feline
    Breed: Siamese
    Age: 6 years
    Weight: 4.8 kg
    Sex: Male Neutered

    History:
    Presenting Complaints:
    - Inappropriate urination (outside litter box)
    - Aggression toward other household cat
    - Excessive vocalization
    - Over-grooming belly area

    Previous Medical History:
    - FLUTD episode in 2022
    - Stress-related cystitis
    - Annual wellness exams normal

    Recent Diagnostics:
    - Urinalysis:
      * USG: 1.045
      * pH: 6.8
      * Crystals: None
      * RBC: 0-2/hpf
    - CBC/Chemistry: All within normal limits
    - Abdominal ultrasound: No abnormalities noted

    Current Medications:
    - Gabapentin 100mg q12h PRN for anxiety
    - Feliway diffuser in home

    Environmental History:
    - Multi-cat household (2 cats)
    - Recent addition of new cat (3 months ago)
    - 3 litter boxes available
    - Vertical space limited

    Owner Notes:
    - Willing to modify environment
    - Interested in behavioral consultation
    - No financial constraints
    - Works long hours
    - Concerned about long-term medication use
    """,
    "medical_records": {},
    "owner_history": {},
    "diagnoses": {},
    "lab_results": {},
    "medications": {},
    "treatment_history": {},
    "results": {},
    "metadata": {},
    "security_flags": {},
    "calculations": {},
    "trends": {},
    "owner_constraints": {},
    "owner_preferences": {}
}

test_case_4 = {
    "messages": [],
    "current_input": """
    Patient Information:
    Name: Charlie
    Species: Canine
    Breed: Labrador Retriever
    Age: 12 years
    Weight: 32.1 kg (down from 34.5 kg 6 months ago)
    Sex: Male Neutered

    History:
    Presenting Complaints:
    - Progressive mobility issues
    - Difficulty rising after rest
    - Increased panting
    - Episodes of night restlessness
    - Decreased appetite

    Previous Medical History:
    - Osteoarthritis diagnosed 3 years ago
    - Hypothyroidism (well-controlled)
    - Stage 2 cardiac disease (diagnosed 1 year ago)

    Recent Diagnostics:
    - CBC/Chemistry:
      * All within normal limits
    - T4: 2.1 µg/dL (1-4)
    - Radiographs:
      * Severe hip and elbow OA
      * Stable cardiac silhouette
    - Blood Pressure: 145/85

    Current Medications:
    - Carprofen 75mg BID
    - Gabapentin 300mg TID
    - Levothyroxine 0.7mg BID
    - Pimobendan 10mg BID

    Treatment History:
    - Adequan injections monthly
    - Physical therapy (weekly)
    - Acupuncture (every 2 weeks)

    Owner Notes:
    - Lives in single-story home
    - Has ramps and non-slip mats
    - Very committed to care
    - Interested in quality of life discussion
    - Can provide medication on schedule
    - Some financial constraints
    """,
    "medical_records": {},
    "owner_history": {},
    "diagnoses": {},
    "lab_results": {},
    "medications": {},
    "treatment_history": {},
    "results": {},
    "metadata": {},
    "security_flags": {},
    "calculations": {},
    "trends": {},
    "owner_constraints": {},
    "owner_preferences": {}
}

In [ ]:
def run_test_cases(app):
    """
    Run all test cases through the workflow
    """
    test_cases = [test_case_1, test_case_2, test_case_3, test_case_4]
    test_case_names = [
        "Senior Cat with Multiple Conditions",
        "Young Dog with Acute Emergency",
        "Middle-Aged Cat with Behavioral Issues",
        "Senior Dog with Chronic Conditions"
    ]

    for i, test_case in enumerate(test_cases):
        print(f"\n{'='*50}")
        print(f"Testing: {test_case_names[i]}")
        print('='*50)

        result = app.invoke(test_case)

        print("\nData Processing Results:")
        print(result["results"]["DataProcessor"])
        print("\nHistory Analysis:")
        print(result["results"]["HistoryAnalyzer"])
        print("\nClinical Analysis:")
        print(result["results"]["ClinicalAnalyzer"])
        print("\nDiagnostic Assessment:")
        print(result["results"]["Diagnostics"])
        print("\nSpecialist Summary:")
        print(result["results"]["SpecialistSummary"])
        print("\nConnections Analysis:")
        print(result["results"]["Connections"])

        print("\nTest Case Completed")
        print('='*50)

run_test_cases(app)


Testing: Senior Cat with Multiple Conditions

Data Processing Results:
**Patient Information:**
- Name: Whiskers
- Species: Feline
- Breed: Maine Coon
- Age: 14 years
- Weight: 5.2 kg (Previous: 6.1 kg 2 months ago)
- Sex: Male Neutered

**History:**
**Presenting Complaints:**
- Progressive weight loss over 2 months
- Increased water consumption
- Vomiting 2-3 times per week
- Decreased grooming
- Lethargy

**Previous Medical History:**
- Dental cleaning and extractions (3 teeth) in 2023
- Chronic kidney disease diagnosed 2022
- UTI treated in 2021

**Recent Diagnostics:**
- CBC/Chemistry (3 days ago):
  - BUN: 45 mg/dL (18-35)
  - Creatinine: 2.8 mg/dL (0.8-2.4)
  - Phosphorus: 6.2 mg/dL (3.4-8.5)
  - T4: 4.8 µg/dL (1.5-4.0)
  - ALT: 85 U/L (10-100)
  - Glucose: 95 mg/dL (71-159)
- Urinalysis:
  - USG: 1.035
  - pH: 6.5
  - Protein: 2+
- Blood Pressure: 165/95 mmHg

**Current Medications:**
- Semintra 4mg/ml (0.25ml PO SID)
- Methimazole 5mg BID
- Renal diet (Hill's k/d)

**Owner Not